# Image processing with Python II

- This is a tutorial on the basics of image analysis with Python

## 1. Importing Modules & Packages

- Import the necesary packages

## 2. Loading Image Data and vizualize

*   Load all images in the /data folder (not one by one, but efficiently using a loop or list comprehension).
*   Transform the GT to binary mask, if necessary
*   Visualize the images using subplots, arranging them in X rows and 2 columns (left column for noisy images and right column for the corresponding ground truth (GT) images).
*   Ensure the color bar range is the same for both columns, i.e., use the same color scale for all images.

## 3. Image filtering

*   Transform RGB images to gray scale, when necessary.
*   Install and use the high-quality denoising method BM3D (Block-Matching and 3D Filtering): https://pypi.org/project/bm3d/.
*   Use Gaussing bluring for denoising too.
*   Use widgets for the interactive selection of the parameters and select the best values for two selected images. In other words, find the parameters that blur out the noise while keeping the structures of interest.
*   Visualize the best results using subplots. For each image, display:
    - The noisy image.
    - The results of the two filtering methods.
*   Ensure the color bar range is the same for both columns, i.e., use the same color scale for all images.
*   Estimate the Blind/Referenceless Image Spatial Quality Evaluator (BRISQUE) (https://pypi.org/project/brisque/) and print the values over the corresponding images.
*   Make a comparative plot of the BRISQUE values (i.e. boxplot) for the raw image, and the results of the two filtering methods.
*   Conclude which one is the best method for denoising in your dataset

## 4. Image Segmentation
*   Sgemnetd teh images after denoising using: manual thresholding, automatic threholding (i.e. Otsu) and adaptative thresholding (i.e. estimated background and substract form the image, then binarize).
*   Use widgets for the interactive selection of the manual threshold and choose the best values for 2 selected images.
*   Visualize the results using subplots. For each image, display:
    - The denoised image.
    - The corresponding mask (GT).
    - The results of the three segmentation methods.
*    Calulate the accuracy, precision, recall, f1 and dice coefficient for all images and print them over them in the plot
*    Make a comparative plots of ecah metric for all the results of the segmentation methods.
*   Conclude which one is the best method for segmentation in your dataset



## 5. Morphological operations

*   Apply differnte morphological operations, i.e. closing, opening and filling holes to the best segmention images obtained in 4.
*   Visualize the best results using subplots. For each image, display:
    - The noisy image.
    - The corresponding ground truth (GT).
    - Best segmented images (from 4).
    - Segmented images after the morphological operations.
*   Calulate the accuracy, precision, recall, f1 and dice coefficient for all image segmentations and print them over them in the plot
*    Make a comparative plots of ecah metric for all the results of the morphological operations.
*   Conclude which one is the best method for your dataset